In [1]:
!pip install python-dotenv
!pip install snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 558.0 kB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 2.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 3.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 825.2 kB/s eta 0:00:000:00:01


In [27]:
from dotenv import load_dotenv
import os

# Cargo mis variables de entorno
load_dotenv()

# Genero el dict de datos para conectarme con Snowflake
credencialesSnowflakeRaw = {
    "sfURL" : os.getenv("SNOWFLAKE_URL"),
    "sfUser" :  os.getenv("SNOWFLAKE_USER"),
    "sfPassword" : os.getenv("SNOWFLAKE_PASSWORD"),
    "sfDatabase" : os.getenv("SNOWFLAKE_DATABASE"),
    "sfSchema" : os.getenv("SNOWFLAKE_SCHEMA_RAW"),
    "sfWarehouse" : os.getenv("SNOWFLAKE_WAREHOUSE"),
    "sfRole" : os.getenv("SNOWFLAKE_ROLE"),
}

print(f"Estas son mis credenciales para Snowflake: {credencialesSnowflakeRaw}")

Estas son mis credenciales para Snowflake: {'sfURL': 'LSNDJXB-RHC82043.snowflakecomputing.com', 'sfUser': 'usuario_spark', 'sfPassword': 'EstudianteEstudiante64', 'sfDatabase': 'NY_TAXI', 'sfSchema': 'RAW', 'sfWarehouse': 'WAREHOUSE_TAXIS', 'sfRole': 'rol_pocos_privilegios'}


In [28]:
import pyspark
from pyspark.sql import SparkSession

# Creo SparkSession para conexión con Snowflake
spark = (SparkSession.builder.appName("IngestaNewYorkTaxis").config("spark.jars.packages", "net.snowflake:snowflake-jdbc:3.13.30,net.snowflake:spark-snowflake_2.12:2.9.0-spark_3.1").getOrCreate())

print(spark)
print("Spark Version : " + spark.version)

# Ejecuto una query de prueba para validar comunicacion con Snowflake
query = "SELECT current_version()"

df = spark.read.format("snowflake").options(**credencialesSnowflakeRaw).option("query", query).load()

df.show()

Spark Version : 3.5.0
+-------------------+
|"CURRENT_VERSION()"|
+-------------------+
|             9.31.0|
+-------------------+



In [29]:
from pyspark.sql import types as T

def crear_tabla_raw_taxis(service: str):
    # Defino el esquema del DataFrame para asegurarme que los datos van a aterrizar correctamente en mi tabla
    
    schema = T.StructType([
        T.StructField("VENDORID", T.IntegerType(), False),
        T.StructField("TPEP_PICKUP_DATETIME", T.TimestampType(), False),
        T.StructField("TPEP_DROPOFF_DATETIME", T.TimestampType(), False),
        T.StructField("PASSENGER_COUNT", T.IntegerType(), True),
        T.StructField("TRIP_DISTANCE", T.FloatType(), True),
        T.StructField("RATECODEID", T.IntegerType(), True),
        T.StructField("STORE_AND_FWD_FLAG", T.StringType(), True),
        T.StructField("PULOCATIONID", T.IntegerType(), False),
        T.StructField("DOLOCATIONID", T.IntegerType(), False),
        T.StructField("PAYMENT_TYPE", T.IntegerType(), True),
        T.StructField("FARE_AMOUNT", T.FloatType(), True),
        T.StructField("EXTRA", T.FloatType(), True),
        T.StructField("MTA_TAX", T.FloatType(), True),
        T.StructField("TIP_AMOUNT", T.FloatType(), True),
        T.StructField("TOLLS_AMOUNT", T.FloatType(), True),
        T.StructField("IMPROVEMENT_SURCHARGE", T.FloatType(), True),
        T.StructField("TOTAL_AMOUNT", T.FloatType(), True),
        T.StructField("CONGESTION_SURCHARGE", T.FloatType(), True),
        T.StructField("RUN_ID", T.StringType(), True),
        T.StructField("SERVICE_TYPE", T.StringType(), True),
        T.StructField("SOURCE_YEAR", T.IntegerType(), True),
        T.StructField("SOURCE_MONTH", T.IntegerType(), True),
        T.StructField("INGESTED_AT_UTC", T.TimestampType(), True),
        T.StructField("SOURCE_PATH", T.StringType(), True)
    ])

    # Creo un DataFrame vacío con el esquema previamente definido por mi
    df_empty = spark.createDataFrame([], schema)

    # Escribo el DataFrame vacío en Snowflake para crear la tabla
    try:
        df_empty.write.format("snowflake") \
            .options(**credencialesSnowflakeRaw) \
            .option("dbtable", f"NY_TAXI_RAW_{service.upper()}") \
            .mode("ignore") \
            .save()

        print(f"Tabla NY_TAXI_RAW_{service.upper()} creada correctamente en Snowflake")
    except Exception as e:
        print(f"Fallo la creacion de la tabla de taxis: {e}")


In [30]:
import datetime

#Hago la presente funcion para generar un identificador unico asociado a cada carga de datos para el RUN_ID 
def generar_run_id():
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

In [31]:
import os 
import requests 
from pyspark.sql.functions import lit, current_timestamp 
from pyspark.sql.types import TimestampType

def ingestar_parquet_a_raw(service: str, year: int, month: int):
    SOURCE_PATH = os.getenv("SOURCE_PATH")
    path_url = f"{SOURCE_PATH}/trip-data/{service}_tripdata_{year}-{month:02d}.parquet"
    local_path = f"/tmp/{service}_tripdata_{year}-{month:02d}.parquet"
    
    # Descargo el archivo Parquet en carpeta temporal para posteriormente leerlo
    try:
        r = requests.get(path_url, stream=True)
        if r.status_code == 200:
            with open(local_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=10000000):
                    f.write(chunk)
        else:
            print(f"Archivo no encontrado en {path_url} (status {r.status_code})")
            return None
    except Exception as e:
        print(f"Error descargando {path_url}: {e}")
        return None
    else:
        print(f"Archivo obtenido exitosamente de: {path_url}")
    
    # Leo el archivo parquet en un df de Spark
    try:
        df = spark.read.parquet(local_path)
    except Exception as e:
        print(f"No se pudo leer {local_path}: {e}")
        return None
    else:
        print(f"Archivo leido exitosamente por Spark: {local_path}")

    run_id = generar_run_id()

    # Elimino columna conflictiva que solo esta presente en unos pocos parquets
    if 'cbd_congestion_fee' in df.columns:
        df = df.drop('cbd_congestion_fee')

    # Homogenizo nombres de columnas y las paso a mayusculas para integridad de tabla
    df = df.withColumnRenamed("LPEP_PICKUP_DATETIME", "TPEP_PICKUP_DATETIME").withColumnRenamed("LPEP_DROPOFF_DATETIME", "TPEP_DROPOFF_DATETIME")
    df = df.toDF(*[c.upper() for c in df.columns])

    # Añado los metadatos indicados en instrucciones de PSET
    df_meta = df.withColumn("run_id", lit(run_id)) \
                .withColumn("service_type", lit(service)) \
                .withColumn("source_year", lit(year)) \
                .withColumn("source_month", lit(month)) \
                .withColumn("ingested_at_utc", current_timestamp()) \
                .withColumn("source_path", lit(path_url))

    
    # Convierto tipos de fecha a Timestamp porque me estaba marcando error al enviar datos al Snowflake sin esta transformacion
    for field in df_meta.schema.fields:
        if field.dataType.typeName() == "timestamp_ntz":
            df_meta = df_meta.withColumn(field.name, df_meta[field.name].cast(TimestampType()))

    conteoFilas = df_meta.count()
    print(f"Ingestando hacia Snowflake {service} {year}-{month}. Total de filas: {conteoFilas}")

    try:
        # Mi logica para idempotencia consiste en por cada run enviar los datos a una tabla temporal luego hacer merge upsert con tabla Raw principal desde tabla temporal y luego borro la tabla temporal
        ingestar_con_tabla_temporal(df_meta, service, run_id)
    except Exception as e2:
        print(f"Error con tabla temporal: {e2}")
        return None

    #Me aseguro de eliminar el archivo parquet temporal
    try:
        os.remove(local_path)
        print(f"Archivo parquet temporal removido: {local_path}")
    except OSError as e:
        print(f"No se pudo remover el archivo parquet temporal {local_path}: {e}")

    #Retorno datos para tabla de conteos de datos consumidos por run
    return {
        "year": year,
        "month": month,
        "count": conteoFilas,
        "run_id": run_id,
        "service_type": service
    }

In [32]:
def ingestar_con_tabla_temporal(df_meta, service: str, run_id: str):

    # Creo tabla temporal en Snowflake
    temp_table_name = f"TEMP_{service.upper()}_{run_id.replace('-', '_')}"
    
    print(f"Creando tabla temporal: {temp_table_name}")
    
    # Escribo datos a tabla temporal en snowflake
    df_meta.write \
        .format("snowflake") \
        .options(**credencialesSnowflakeRaw) \
        .option("dbtable", temp_table_name) \
        .mode("overwrite") \
        .save()
    
    # Ejecuto Merge para idempotencia
    try:
        procesarTablaTempToRawTaxis(temp_table_name, service)
    except Exception as e4:
        print(f"Error en Merge: {e4}")
        raise e4
    else:
        print(f"Merge ejecutado exitosamente desde tabla temporal")

In [33]:
import snowflake.connector

def procesarTablaTempToRawTaxis(temp_table_name: str, service: str):

    #Esta sera mi query merge la cual verifica similitud con datos de clave natural (vendorID + timestamps + PU/DO) en caso de similitud hace update y sino hace insert de datos. Asi aseguro idempotencia y evito duplicados
    merge_query = f"""
    MERGE INTO {credencialesSnowflakeRaw['sfDatabase']}.{credencialesSnowflakeRaw['sfSchema']}.NY_TAXI_RAW_{service.upper()} AS target
    USING {credencialesSnowflakeRaw['sfDatabase']}.{credencialesSnowflakeRaw['sfSchema']}.{temp_table_name} AS source
    ON target.VENDORID = source.VENDORID 
       AND target.TPEP_PICKUP_DATETIME = source.TPEP_PICKUP_DATETIME 
       AND target.TPEP_DROPOFF_DATETIME = source.TPEP_DROPOFF_DATETIME
       AND target.PULOCATIONID = source.PULOCATIONID
       AND target.DOLOCATIONID = source.DOLOCATIONID
    WHEN MATCHED THEN
        UPDATE SET
            target.PASSENGER_COUNT = source.PASSENGER_COUNT,
            target.TRIP_DISTANCE = source.TRIP_DISTANCE,
            target.RATECODEID = source.RATECODEID,
            target.STORE_AND_FWD_FLAG = source.STORE_AND_FWD_FLAG,
            target.PAYMENT_TYPE = source.PAYMENT_TYPE,
            target.FARE_AMOUNT = source.FARE_AMOUNT,
            target.EXTRA = source.EXTRA,
            target.MTA_TAX = source.MTA_TAX,
            target.TIP_AMOUNT = source.TIP_AMOUNT,
            target.TOLLS_AMOUNT = source.TOLLS_AMOUNT,
            target.IMPROVEMENT_SURCHARGE = source.IMPROVEMENT_SURCHARGE,
            target.TOTAL_AMOUNT = source.TOTAL_AMOUNT,
            target.CONGESTION_SURCHARGE = source.CONGESTION_SURCHARGE,
            target.RUN_ID = source.RUN_ID,
            target.SERVICE_TYPE = source.SERVICE_TYPE,
            target.SOURCE_YEAR = source.SOURCE_YEAR,
            target.SOURCE_MONTH = source.SOURCE_MONTH,
            target.INGESTED_AT_UTC = source.INGESTED_AT_UTC,
            target.SOURCE_PATH = source.SOURCE_PATH
    WHEN NOT MATCHED THEN
        INSERT (
            VENDORID, TPEP_PICKUP_DATETIME, TPEP_DROPOFF_DATETIME, PASSENGER_COUNT, 
            TRIP_DISTANCE, RATECODEID, STORE_AND_FWD_FLAG, PULOCATIONID, DOLOCATIONID, 
            PAYMENT_TYPE, FARE_AMOUNT, EXTRA, MTA_TAX, TIP_AMOUNT, TOLLS_AMOUNT, 
            IMPROVEMENT_SURCHARGE, TOTAL_AMOUNT, CONGESTION_SURCHARGE, 
            RUN_ID, SERVICE_TYPE, SOURCE_YEAR, SOURCE_MONTH, INGESTED_AT_UTC, SOURCE_PATH
        ) VALUES (
            source.VENDORID, source.TPEP_PICKUP_DATETIME, source.TPEP_DROPOFF_DATETIME, 
            source.PASSENGER_COUNT, source.TRIP_DISTANCE, source.RATECODEID, 
            source.STORE_AND_FWD_FLAG, source.PULOCATIONID, source.DOLOCATIONID, 
            source.PAYMENT_TYPE, source.FARE_AMOUNT, source.EXTRA, source.MTA_TAX, 
            source.TIP_AMOUNT, source.TOLLS_AMOUNT, source.IMPROVEMENT_SURCHARGE, 
            source.TOTAL_AMOUNT, source.CONGESTION_SURCHARGE, 
            source.RUN_ID, source.SERVICE_TYPE, source.SOURCE_YEAR, source.SOURCE_MONTH, 
            source.INGESTED_AT_UTC, source.SOURCE_PATH
        )
    """

    #query para que posterior al merge se haga drop de tabla temporal
    drop_query= f"""DROP TABLE IF EXISTS {credencialesSnowflakeRaw['sfDatabase']}.{credencialesSnowflakeRaw['sfSchema']}.{temp_table_name}"""  

    try:
        conn = snowflake.connector.connect(
        user=os.getenv("SNOWFLAKE_USER"),
        password=os.getenv("SNOWFLAKE_PASSWORD"),
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
        warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
        database=os.getenv("SNOWFLAKE_DATABASE"),
        schema=os.getenv("SNOWFLAKE_SCHEMA_RAW")
        )
    
        cur = conn.cursor()

        #Hago la conexion con snowflake y ejecuto la query de merge y luego la de drop
        cur.execute(merge_query)
        cur.execute(drop_query)
    
        conn.commit()
        cur.close()
        conn.close()
        
    except Exception as e3:
        print(f"Fallo la migración de datos de la tabla temporal a la tabla Raw de Taxis {service}: {e3}")
        raise e3
    else:
        print(f"Se migro exitosamente los datos de la tabla temporal a la principal Raw de Taxis {service}") 
    

In [34]:
import json
import os
import requests

#Defino funciones tipicas de checkpoint para en caso de fallo no ingestar datos desde cero
def save_checkpoint(year, month):
    with open(CHECKPOINT_FILE, "w") as f:
        json.dump({"year": year, "month": month}, f)

def load_checkpoint(CHECKPOINT_FILE):
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r") as f:
            return json.load(f)
    return {"year": 0, "month": 0}

#Genero arreglo que contendra datos de ingesta para tabla de conteos
resultadosGeneralesIngesta=[]

try:
    #Leo los datos de tipos de taxis, years y months desde variables de entorno
    tipos_taxis=os.getenv("SERVICES").split(',')
    for tipo_taxi in tipos_taxis:
        
        lista_years = sorted([int(item) for item in (os.getenv("YEARS").split(','))])
        lista_months = sorted([int(item) for item in (os.getenv("MONTHS").split(','))])
        #Cargo el checkpoint y en caso de que tenga registros recorto los arreglos de months y years para recorrer desde ultima ingesta exitosa
        CHECKPOINT_FILE = f"checkpointTaxis{tipo_taxi.capitalize()}.json"
        print(CHECKPOINT_FILE)
        checkpoint=load_checkpoint(CHECKPOINT_FILE)
        print(f"checkpoint: {checkpoint}")
        
        if ( checkpoint != {"year": 0, "month": 0} and (int(checkpoint["month"]) in lista_months) and (int(checkpoint["year"]) in lista_years)):    
            if ( int(checkpoint["month"]) == lista_months[-1] and int(checkpoint["year"]) != lista_years[-1] ):
                lista_years= lista_years[lista_years.index(checkpoint["year"])+1:]
            elif ( int(checkpoint["month"]) == lista_months[-1] and int(checkpoint["year"]) == lista_years[-1] ): 
                continue
            else:
                lista_years= lista_years[lista_years.index(checkpoint["year"]):]
                lista_months= lista_months[lista_months.index(checkpoint["month"])+1:]

        crear_tabla_raw_taxis(tipo_taxi)
        
        for year_taxi in lista_years:
            for month_taxi in lista_months:  
                #Llamo a la funcion de ingesta de datos iterativamente para cada mes, year y tipo de taxi
                print(f"Iniciando ingesta de datos de taxis {tipo_taxi}: {month_taxi}-{year_taxi}")
                resultadosParciales=ingestar_parquet_a_raw(tipo_taxi, year_taxi, month_taxi)
                #Guardo los resultados y genero el checkpint
                if (resultadosParciales != None):
                    resultadosGeneralesIngesta.append(resultadosParciales)
                    save_checkpoint(year_taxi,month_taxi)
                
except Exception as e5:
    #Como en todas las funciones vistas hago manejo de errores
    print(f"Fallo el proceso de ingesta masiva de datos de taxis NY: {e5}")
else:
    print("El proceso de ingesta masiva de taxis NY fue exitoso")

checkpointTaxisYellow.json
checkpoint: {'year': 0, 'month': 0}
Tabla NY_TAXI_RAW_YELLOW creada correctamente en Snowflake
Iniciando ingesta de datos de taxis yellow: 1-2015
Archivo obtenido exitosamente de: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet
Archivo leido exitosamente por Spark: /tmp/yellow_tripdata_2015-01.parquet
Ingestando hacia Snowflake yellow 2015-1. Total de filas: 12741035
Creando tabla temporal: TEMP_YELLOW_20251009_173030
Se migro exitosamente los datos de la tabla temporal a la principal Raw de Taxis yellow
Merge ejecutado exitosamente desde tabla temporal
Archivo parquet temporal removido: /tmp/yellow_tripdata_2015-01.parquet
Iniciando ingesta de datos de taxis yellow: 2-2015
Archivo obtenido exitosamente de: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-02.parquet
Archivo leido exitosamente por Spark: /tmp/yellow_tripdata_2015-02.parquet
Ingestando hacia Snowflake yellow 2015-2. Total de filas: 12442394
Cr

In [35]:
print(resultadosGeneralesIngesta)

[{'year': 2015, 'month': 1, 'count': 12741035, 'run_id': '20251009_173030', 'service_type': 'yellow'}, {'year': 2015, 'month': 2, 'count': 12442394, 'run_id': '20251009_173710', 'service_type': 'yellow'}, {'year': 2015, 'month': 3, 'count': 13342951, 'run_id': '20251009_174342', 'service_type': 'yellow'}, {'year': 2015, 'month': 4, 'count': 13063758, 'run_id': '20251009_175009', 'service_type': 'yellow'}, {'year': 2015, 'month': 5, 'count': 13157677, 'run_id': '20251009_175705', 'service_type': 'yellow'}, {'year': 2015, 'month': 6, 'count': 12324936, 'run_id': '20251009_180330', 'service_type': 'yellow'}, {'year': 2015, 'month': 7, 'count': 11559666, 'run_id': '20251009_180918', 'service_type': 'yellow'}, {'year': 2015, 'month': 8, 'count': 11123123, 'run_id': '20251009_181449', 'service_type': 'yellow'}, {'year': 2015, 'month': 9, 'count': 11218122, 'run_id': '20251009_182100', 'service_type': 'yellow'}, {'year': 2015, 'month': 10, 'count': 12307333, 'run_id': '20251009_182630', 'serv